# Puente estrecho 

### Peralta Rionda Gabriel Zadquiel 
Implementación de la solución del problema del puente estrecho.

In [5]:
from time import sleep
from random import random
from multiprocessing import Process
from multiprocessing import Semaphore

A continuación veremos un primer intento de implementación de la solución del problema del puente estrecho con semáforos, en este se utilizan dos semáforos, un semáforo binario y otro entero que se encargarán de regular el tránsito de los automoviles por el puente.

In [6]:
def carrito(semaphore1,semaphore2,i):
  with semaphore2: # Se llama al semáforo entero, que controlara el acceso de tres primero coches que 
  #quieran ingresar al puente
      sleep(random())
      if i%2 == 0: # Si el identificador del carrito es par este carrito vendra del lado derecho.
        with semaphore1: #Se llama al segundo semáforo para que este se encargue de controlar el flujo del puente 
        #Y varios carritos no esten intentando acceder al puente, y se les permitira hasta que un carrito haya terminado de cruzar.
          print(f'Soy el carrito {i}')
          print('Estoy en el puente del lado derecho')
          print('Estoy cruzando el puente')
          print('Sali del puente')
      else: # Si el identificador del carrito es impar este carrito vendra del lado izquierdo.
        with semaphore1: #Se llama al segundo semáforo para que este se encargue de controlar el flujo del puente 
        #Y varios carritos no esten intentando acceder al puente, y se les permitira hasta que un carrito haya terminado de cruzar.
          print(f'Soy el carrito {i}')
          print('Estoy en el puente del lado izquierdo')
          print('Estoy cruzando el puente')
          print('Sali del puente')

if __name__ == '__main__':
    semaphore1 = Semaphore(1) #Se crean los dos semaforos.
    semaphore2 = Semaphore(3)
    processes = [Process(target=carrito, args=(semaphore1,semaphore2, i)) for i in range(6)] #se crean 6 carritos
    #Estos carritos lo que harán será conducir y con el control del semaforo evitaremos colisiones.
    for process in processes:
        process.start()
    for process in processes:
        process.join()

Soy el carrito 1
Estoy en el puente del lado izquierdo
Estoy cruzando el puente
Sali del puente
Soy el carrito 0
Estoy en el puente del lado derecho
Estoy cruzando el puente
Sali del puente
Soy el carrito 3
Estoy en el puente del lado izquierdo
Estoy cruzando el puente
Sali del puente
Soy el carrito 2
Estoy en el puente del lado derecho
Estoy cruzando el puente
Sali del puente
Soy el carrito 4
Estoy en el puente del lado derecho
Estoy cruzando el puente
Sali del puente
Soy el carrito 5
Estoy en el puente del lado izquierdo
Estoy cruzando el puente
Sali del puente


La implementación anterior controla bien el trafico de los carritos ya que conforme van llegando los carritos estos pueden ir pasando, el problema es que estamos limitando el puente a que solo un carrito pueda estar pasando el puente, esto lo hace bien sin importar el número de carritos que quieran acceder, se esperan conforme van llegando y el semáforo se encarga de que vayan pasando uno por uno.  
Por eso a continuación presentamos una segunda solución al problema la cual se encarga de solucionar esta pequeña inconveniencia que tenemos:

In [3]:
import threading
import time
import multiprocessing as mp
import sys

#Creamos nuestra propia clase semáforo que es necesaria para tener un buen control del transito
#ya que en python los semaphoros no tienen ciertas herramientas que necesitamos así como el 
#control principal de las funciones wait y signal en el semáforo.
class Semaphore():
    def __init__(self, initial):
        self.lock = threading.Condition() 
        self.value = initial

    def up(self):
        with self.lock:
            self.value += 1
            self.lock.notify() 

    def down(self):
        with self.lock:
            while self.value == 0:
                self.lock.wait() 
            self.value -= 1
#Creamos el puente el que se encarfara de estar controlando y administrando el número de carros que pasaran por este
#Notemos que a diferencia del primer intento aquí estamos tomando control de la situación que necesitabamos.  
class Puente():
    def __init__(self):    
        self.lock = threading.Condition()
        self.taken = False

    def take(self, sentido):#esta función la usamos para la sincronización
        with self.lock:
            while self.taken == True:
                sys.stdout.write("Carro[%s] esperando por el puente \n" % (sentido))
                self.lock.wait()
            self.taken = True
            sys.stdout.write("Carro[%s] pasando por el puente \n"%(sentido))
            sys.stdout.write(" Carro[%s] pasó por el puente \n"%(sentido))
            self.lock.notifyAll()

    def drop(self, sentido):#Lo usamos para la sincronización
        with self.lock:
            while self.taken == False:
                self.lock.wait()
            self.taken = False
            self.lock.notifyAll()
            
class Automovil_izq(threading.Thread): #Creamos la clase de coches izquierdos.
    def __init__(self, left, semaforo):
        threading.Thread.__init__(self)
        self.left = left
        self.semaforo = semaforo
    
    def run(self): #Este método será el encargado de que el auto pase por el puente
        while True:
            sys.stdout.write("Carro sentido izquierdo quiere pasar\n" )
            time.sleep(1)
            self.semaforo.down()              
            time.sleep(1)
            self.left.take('izquierdo')        
            time.sleep(1)  
            self.left.drop('izquierdo')  
            time.sleep(1)
            self.semaforo.up()             
            time.sleep(1)
            
class Automovil_der(threading.Thread): #Creamos la clase de automoviles derechos.
    def __init__(self, right, semaforo):
        threading.Thread.__init__(self)
        self.right = right
        self.semaforo = semaforo
    
    def run(self): #Este método es el encargado de que pase por el puente.
        for i in range(1):
            sys.stdout.write("Carro sentido derecho quiere pasar\n")
            time.sleep(1)
            self.semaforo.down()              
            time.sleep(1)
            self.right.take('derecho')        
            time.sleep(1)  
            self.right.drop('derecho')  
            time.sleep(1)
            self.semaforo.up()             
            time.sleep(1)

if __name__ == '__main__':
    n_izq = 5 
    n_der = 5
    c = [Puente() for i in range(n_izq + n_der)]
    
    semaforo = Semaphore(2)
    sys.stdout.write("\n")
    p_izq = [Automovil_izq(c[i], semaforo) for i in range(n_izq)] #Creamos los autos izquierdos
    p_der = [Automovil_der(c[i], semaforo) for i in range(n_der)] #Creamos los autos derechos

    for i in range(n_izq):
        p_izq[i].start()
    for i in range(n_der):
        p_der[i].start()
        
    for i in range(n_izq):
        p_der[i].join()
    for i in range(n_der):
        p_izq[i].join()
    



Carro sentido izquierdo quiere pasar
Carro sentido izquierdo quiere pasar
Carro sentido izquierdo quiere pasar
Carro sentido izquierdo quiere pasar
Carro sentido izquierdo quiere pasar
Carro sentido derecho quiere pasar
Carro sentido derecho quiere pasar
Carro sentido derecho quiere pasar
Carro sentido derecho quiere pasar
Carro sentido derecho quiere pasar
Carro[izquierdo] pasando por el puente 
 Carro[izquierdo] pasó por el puente 
Carro[izquierdo] pasando por el puente 
 Carro[izquierdo] pasó por el puente 
Carro[izquierdo] pasando por el puente 
Carro[izquierdo] pasando por el puente 
 Carro[izquierdo] pasó por el puente 
 Carro[izquierdo] pasó por el puente 
Carro sentido izquierdo quiere pasar
Carro sentido izquierdo quiere pasar
Carro sentido izquierdo quiere pasar
Carro sentido izquierdo quiere pasar
Carro[izquierdo] pasando por el puente 
 Carro[izquierdo] pasó por el puente 
Carro[derecho] pasando por el puente 
 Carro[derecho] pasó por el puente 
Carro sentido izquierdo qui

KeyboardInterrupt: ignored